In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.colors as mcolors
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Polygon
import missingno as msno
import os
import wget
import openpyxl
import math

In [2]:
#wget.download("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip")

'cb_2018_us_state_500k.zip'

(unzip the file)

In [2]:
gdf = gpd.read_file(os.getcwd()+'/cb_2018_us_state_500k')
gdf.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169..."


In [3]:
file_path = 'D:/opioids/opioids/PDMP.dta'
PDMP = pd.read_stata(file_path)
PDMP = PDMP[['statefip','statecode','statename','mopdmp_date','mopdmp_year','mopdmp_month']]
print(PDMP.head())

   statefip statecode   statename mopdmp_date  mopdmp_year  mopdmp_month
0       2.0        AK      Alaska  2012-01-01       2012.0           1.0
1       1.0        AL     Alabama  2006-04-01       2006.0           4.0
2       5.0        AR    Arkansas  2013-05-01       2013.0           5.0
3       4.0        AZ     Arizona  2008-12-01       2008.0          12.0
4       6.0        CA  California  2009-09-01       2009.0           9.0


In [4]:
start_date = "1999-06-01"
end_date = "2017-02-01"
date_array = pd.date_range(start=start_date, end=end_date, freq='MS')
print(date_array)

DatetimeIndex(['1999-06-01', '1999-07-01', '1999-08-01', '1999-09-01',
               '1999-10-01', '1999-11-01', '1999-12-01', '2000-01-01',
               '2000-02-01', '2000-03-01',
               ...
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01'],
              dtype='datetime64[ns]', length=213, freq='MS')


In [5]:
for i in range(len(date_array)):
    new_column_name = f'Column_{i+1}'
    PDMP[new_column_name] = 1*(PDMP.mopdmp_date < date_array[i])


C:\Users\guill\AppData\Local\Temp\ipykernel_22664\348320381.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  PDMP[new_column_name] = 1*(PDMP.mopdmp_date < date_array[i])
C:\Users\guill\AppData\Local\Temp\ipykernel_22664\348320381.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  PDMP[new_column_name] = 1*(PDMP.mopdmp_date < date_array[i])
C:\Users\guill\AppData\Local\Temp\ipykernel_22664\348320381.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

In [6]:
gdf = gdf.merge(PDMP,left_on='STUSPS',right_on='statecode')

from: https://medium.com/@alex_44314/use-python-geopandas-to-make-a-us-map-with-alaska-and-hawaii-39a9f5c222c6

In [7]:
def makeColorColumn(gdf,variable,vmin,vmax):
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax, clip=True)
    mapper = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.YlOrBr)
    gdf['value_determined_color'] = gdf[variable].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    return gdf

In [ ]:
for i in range(len(date_array)):
    variable = f'Column_{i+1}'

    man_date = f'{date_array[i]}'
    man_date = man_date.replace(' 00:00:00', '')

    vmin, vmax = gdf.Column_150.min(), gdf.Column_150.max() 
    colormap = "YlOrBr"
    gdf = makeColorColumn(gdf,variable,vmin,vmax)

    visframe = gdf.to_crs({'init':'epsg:2163'})

    fig, ax = plt.subplots(1, figsize=(18, 14))
    ax.axis('off')

    hfont = {'fontname':'Helvetica'}

    ax.set_title(f'Modern PDMP System Operational: {man_date}', **hfont, fontdict={'fontsize': '42', 'fontweight' : '1'})

    sm = plt.cm.ScalarMappable(cmap=colormap, \
                    norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []
    tick_font_size = 16
    ax.annotate("Data:  Horwitz et al. (2018)", xy=(0.22, .085), xycoords='figure fraction', fontsize=14, color='#555555')

    for row in visframe.itertuples():
        if row.statecode not in ['AK','HI']:
            vf = visframe[visframe.statecode==row.statecode]
            c = gdf[gdf.statecode==row.statecode][0:1].value_determined_color.item()
            vf.plot(color=c, linewidth=0.8, ax=ax, edgecolor='0.8')


    akax = fig.add_axes([0.1, 0.17, 0.2, 0.19])   
    akax.axis('off')
    polygon = Polygon([(-170,50),(-170,72),(-140, 72),(-140,50)])
    alaska_gdf = gdf[gdf.statecode=='AK']
    alaska_gdf.clip(polygon).plot(color=gdf[gdf.statecode=='AK'].value_determined_color, linewidth=0.8,ax=akax, edgecolor='0.8')

    hiax = fig.add_axes([.28, 0.20, 0.1, 0.1])   
    hiax.axis('off')
    hipolygon = Polygon([(-160,0),(-160,90),(-120,90),(-120,0)])
    hawaii_gdf = gdf[gdf.statecode=='HI']
    hawaii_gdf.clip(hipolygon).plot(column=variable, color=hawaii_gdf['value_determined_color'], linewidth=0.8,ax=hiax, edgecolor='0.8')

    fig.savefig(f'D:/opioids/plots_pdmps/{i}.png',dpi=400, bbox_inches="tight")
    plt.close()

en la cmd: (need to install ImageMagick -> en el .exe, select old features (e.g. convert))

C:\Users\guill>cd C:\Users\guill\Documents\Scripts\opioids\plots_pdmps

C:\Users\guill\Documents\Scripts\opioids\plots_pdmps>convert -delay 10 -loop 0 *.png anim.gif